## Make sure to run Chunking Series and Durations first. This will not be accurate if those files are not up to date.

In [1]:
import pandas as pd
import numpy as np

In [2]:
scaleColumns = ['EntryFeeAmount', 'TotalPrizeAmount','MaxNumberPlayers', 'MaxEntriesPerUser', 'DistinctUsers', 'NumGames', 'DraftablePlayersInSet','PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition', 'Duration']
categoricalColumns = ['SportName', 'VariantName', 'Contest_Group']

In [3]:
rawDF = pd.read_csv('data/WorkingData.csv')
rawDF = pd.merge(rawDF, pd.read_csv('data/Durations.csv').set_index('ContestId'), on='ContestId', how='left')

In [79]:
processedDF = rawDF.copy(deep=True).set_index('ContestId')

In [80]:
metaColumns = []
successCol = []


processedDF['Success'] = (processedDF['MaxNumberPlayers'] == processedDF['Entries'])
successCol.append('Success')

successDF = processedDF[['Success']]

for scol in scaleColumns:
    processedDF[scol+"_Scaled"] = processedDF[scol]/processedDF[scol].max()
    metaColumns.append(scol+"_Scaled")
    
for scol in categoricalColumns:
    for val in processedDF[scol].unique():
        catStr = scol+"_"+val
        processedDF[catStr] = (processedDF[scol]==val)
        metaColumns.append(catStr)
        
metaDF = processedDF[metaColumns]

In [153]:
def renamePredColumns(df, prefix):
    renameDict = {}
    columns = []
    for col in df.columns:
        if(col[:4]=='Pred'):
            colName = "Pred_"+prefix+"_"+col[4:]
            renameDict[col] = colName
            columns.append(colName)
    df = df.rename(columns=renameDict)
    return df



In [152]:
def renameABColumns(df, prefix):
    renameDict = {}
    columns = []
    for col in df.columns:
        colName = col
        version = 'default'
        if(col[-1:][0]=='A'):
            version = col[:-1]
        elif(col[0] == 'A'):
            version = col[1:]
        
        if(not (version == 'default')):
            newName = "A_"+prefix+"_"+version
            renameDict[colName] = newName
            renameDict[colName.replace('A', 'B')] = newName.replace('A', 'B')
            columns.append(newName)
            columns.append(newName.replace('A', 'B'))
    df = df.rename(columns=renameDict)
    return df


In [90]:
def getABDF(dfs):
    for df in dfs:
        print(len(df))
getABDF([lrDF, kfDF])

649329
649329


### Raw

In [149]:
lrDF = pd.read_csv('data/LR_Preds.csv').set_index('ContestId').drop(columns=['Duration', 'Unnamed: 0'])
lrDF = renamePredColumns(lrDF, 'LR')
lrDF = renameABColumns(lrDF, 'LR')

kfDF = pd.read_csv('data/KF_Preds.csv').set_index('ContestId').drop(columns=['Duration'])
kfDF = renamePredColumns(kfDF, 'KF')
kfDF = renameABColumns(kfDF, 'KF')

In [150]:
predDF = pd.merge(rawDF, kfDF, on='ContestId', how='left')
predDF = pd.merge(predDF, lrDF, on='ContestId', how='left')
predDF = predDF.set_index('ContestId')
# predDF = pd.merge(predDF, lrDF, on='ContestId', how='left')

In [151]:
kfColumns = []
lrColumns = []

for col in kfDF.columns:
    if(col[4:]=='Pred'):
        label = col+"_Scaled"
        predDF[label] = predDF[col]/predDF["MaxNumberPlayers"]
        kfColumns.append(label)
    else:
        kfColumns.append(col)

kfPredsDF = predDF[kfColumns]
        
for col in lrDF.columns:
    if(col[4:]=='Pred'):
        label = col+"_Scaled"
        predDF[label] = predDF[col]/predDF["MaxNumberPlayers"]
        lrColumns.append(label)
    else:
        lrColumns.append(col)
        
lrPredsDF = predDF[lrColumns]

### Scaled

In [154]:
lrsDF = pd.read_csv('data/LR_Scaled_Preds.csv').set_index('ContestId')
lrsDF = renamePredColumns(lrsDF, 'LRS')
lrsDF = renameABColumns(lrsDF, 'LRS')


kfsDF = pd.read_csv('data/KF_Scaled_Preds.csv').set_index('ContestId')
kfsDF = renamePredColumns(kfsDF, 'KFS')
kfsDF = renameABColumns(kfsDF, 'KFS')

In [155]:
predDF = pd.merge(rawDF, lrsDF, on='ContestId', how='left')
predDF = pd.merge(predDF, kfsDF, on='ContestId', how='left')
predDF = predDF.set_index('ContestId')

In [156]:
kfColumns = []
lrColumns = []

for col in kfsDF.columns:
    if(col[4:]=='Pred'):
        label = col+"_Scaled"
        predDF[label] = predDF[col]/100
        kfsColumns.append(label)
    else:
        kfColumns.append(col)

kfsPredsDF = predDF[kfColumns]
        
for col in lrsDF.columns:
    if(col[4:]=='Pred'):
        label = col+"_Scaled"
        predDF[label] = predDF[col]/100
        lrColumns.append(label)
    else:
        lrColumns.append(col)
        
lrsPredsDF = predDF[lrColumns]

In [158]:
kfPredsDF.to_csv('data/KF_Results.csv')
lrPredsDF.to_csv('data/LR_Results.csv')
kfsPredsDF.to_csv('data/KF_Scaled_Results.csv')
lrsPredsDF.to_csv('data/LR_Scaled_Results.csv')
metaDF.to_csv('data/MetaData.csv')
# abDF.to_csv('data/AB_Values.csv')

In [159]:
workedData = pd.merge(metaDF, kfPredsDF, on='ContestId', how='left')
workedData = pd.merge(workedData, lrPredsDF, on='ContestId', how='left')
workedData = pd.merge(workedData, kfsPredsDF, on='ContestId', how='left')
workedData = pd.merge(workedData, lrsPredsDF, on='ContestId', how='left')
# workedData = pd.merge(workedData, abDF, on='ContestId', how='left')
workedData = pd.merge(successDF, workedData, on='ContestId', how='left')

In [160]:
for c in workedData.columns:
    print(c)

Success
EntryFeeAmount_Scaled
TotalPrizeAmount_Scaled
MaxNumberPlayers_Scaled
MaxEntriesPerUser_Scaled
DistinctUsers_Scaled
NumGames_Scaled
DraftablePlayersInSet_Scaled
PaidUsersInDraftGroup_Scaled
TopPrize_Scaled
MaxPayoutPosition_Scaled
Duration_Scaled
SportName_NAS
SportName_CFB
SportName_MLB
SportName_NFL
SportName_SOCC
SportName_PGA
SportName_MMA
SportName_LOL
SportName_NHL
SportName_NBA
SportName_SOC
SportName_CBB
SportName_GOLF
SportName_CFL
SportName_EL
SportName_TEN
VariantName_Classic
VariantName_Classic (Old)
VariantName_Non Late Swap
VariantName_11 Man
VariantName_Weekend
VariantName_Arcade
VariantName_WNBA
VariantName_Tiers
VariantName_Playoff Arcade
VariantName_Tiers Non Late Swap
VariantName_Final Series
VariantName_Showdown
VariantName_Match Play
VariantName_Showdown Captain Mode
VariantName_Lowball
VariantName_Single Match
VariantName_Tiers: Man of the Match
VariantName_Tiers: Playmakers
VariantName_Tiers: Draft Pick
VariantName_Tiers: College Conference
VariantName_In

In [161]:
workedData.to_csv('data/WorkedData.csv')